In [12]:
import pandas as pd
import numpy as np

In [13]:
pd.set_option('display.max_columns', None)

In [14]:
# 가격 데이터 세팅
test_df = pd.read_csv("./test/test_df.csv")
test_df = test_df.set_index('date')

In [15]:
# Not Working Period 근거 찾을 df
df = pd.read_csv("./csv/preprocessed_index.csv")
df = df.set_index('date')
df['close'] = test_df['close']
df['open'] = test_df['open']
df['high'] = test_df['high']
df['low'] = test_df['low']
df['high_low'] = test_df['high_low']
df['volume'] = test_df['volume']

df = df.dropna()

# 컬럼명 변환
df.columns = df.columns.str.lower().str.replace(' ', '_')

##### 데이터 명세서

n yr+ hodl wave = 온체인에서 N년동안 움직이지 않은 비트코인이 전체 비트코인 중 차지하는 비율
<br>-> 분석 상 주의해야 할 부분 (단순히 Holding이 아니라 잃어버린 비트코인일 가능성)

addresses with > n btc = N개의 비트코인 초과를 가지고 있는 Address의 수

balanced_price = Transferred Price - Realized Price.
<br>Transfered Price = Sum of CDD / Existing Supply of Bitcoin in circulation
<br>Realized Price = the cost-basis value of all bitcoin in circulation


active_addresses_7dma = 당일 온체인에서 Receive, Send한 Address의 수 (7일 이동평균)

hash_rate_7dma = 비트코인 Hash Rate (7일 이동평균)

miner_difficulty = 비트코인 채굴 난이도

circulating_supply = 실제로 시장에 유통중인 비트코인의 수 (장기보유, 분실 제외한 비트코인 수)

cdd_(90dma) = Coin Destroyed Days, 코인이 얼마나 오랫동안 보관되었다가 이동했는지를 알려줌 (보관량 X 보관일수)

cvdd = Cumulative value of CDD (Cumulative CDD 가중 비트코인 가격)

Delta Cap = Realized Cap - Average Cap (대충 비트코인 가격 상한선 지표)

long-term_holder_realized_price = 장기보유자 realized price

short-term_holder_realized_price = 단기보유자 realized price

miner_revenue_-_block_rewards = 채굴자 순보상

miner_revenue_-_fees = 채굴자 보상 수수료

miner_fees_% = 채굴자 보상 수수료 비율

miner_revenue_-_total = 채굴자 순보상 + 채굴자 보상 수수료

new_addresses = 신규 지갑주소 수

addresses_in_loss = 손해인 지갑 비율

addresses_in_profit = 수익인 지갑 비율

realized_price = the cost-basis value of all bitcoin in circulation

mvocd = 소멸된 코인의 중앙값(일)

supply_adjusted_cdd_(90dma) = 공급량 조정 CDD

terminal_price = Transferred price / supply of bitcoin in circulation * 21 (21 곱하는 이유는 비트코인양 상한이 2100만개)

top_cap = average cap * 20 (대충 비트코인 가격 상한선 지표)

vdd_multiple = VDD / 30 day average VDD | VDD = CDD * Coin Price

whale_shadows = 한 지갑에 오래 보관된 비트코인 중에 옮겨간 양

close = 종가

open = 시가

high = 고가

low = 저가

high_low = 고가 - 저가

volume = 거래량

### 목표
<br>안 먹히는 기간
<br>2018.01.01 - 2018.12.31
<br>2020.01.01 - 2020.12.31
<br>2022.01.01 - 2022.12.31
<br>2024.01.01 - 2024.12.31

<br>되는 기간
<br>2017.01.01 - 2017.12.31
<br>2019.01.01 - 2019.12.31
<br>2021.01.01 - 2021.12.31
<br>2023.01.01 - 2023.12.31


<br>왜 안 먹히는지 이유 찾기
<br>예시: New address / activation addresses 보니까 새로운 뉴비 유입이 많았어서 그렇다 ...~~

In [17]:
df

,10yr+_hodl_wave,1yr+_hodl_wave,5yr+_hodl_wave,addresses_with_>_0_btc,addresses_with_>_0.01_btc,addresses_with_>_0.1_btc,addresses_with_>_1_btc,addresses_with_>_10_btc,addresses_with_>_100_btc,"addresses_with_>_1,000_btc","addresses_with_>_10,000_btc",balanced_price,active_addresses_7dma,hash_rate_7dma,miner_difficulty,circulating_supply,cdd_(90dma),cvdd,delta_top,long-term_holder_realized_price,miner_revenue_-_block_rewards,miner_revenue_-_fees,miner_fees_%,miner_revenue_-_total,new_addresses,addresses_in_loss,addresses_in_profit,realized_price,mvocd,short-term_holder_realized_price,supply_adjusted_cdd_(90dma),terminal_price,top_cap,vdd_multiple,whale_shadows,close,open,high,low,high_low,volume
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-01,0.000000,0.585430,0.173538,13306988,3773175,1539437,548895,147877,18258,1740,112,330.544015,606551.714286,2.398541e+06,3.176884e+11,1.607735e+07,9.314657e+06,275.039383,1714.757520,316.328863,1950.000,65.079499,0.032296,2015.079499,213250,0.004492,0.995508,433.187437,462.667890,755.695137,0.582031,2155.511858,6587.772695,1.585406,269.205418,4285.08,4261.48,4485.39,4200.74,284.65,795.150377
2017-01-02,0.000000,0.584720,0.173592,13311199,3757099,1529263,546393,147680,18253,1738,112,332.567571,622866.285714,2.444491e+06,3.176884e+11,1.607934e+07,9.357013e+06,275.992683,1729.303620,316.335780,1987.500,112.884147,0.053745,2100.384147,350945,0.004902,0.995098,435.554029,462.667890,762.673991,0.584572,2162.715620,6597.872918,1.491400,217.542587,4108.37,4285.08,4371.52,3938.77,432.75,1199.888264
2017-01-03,0.000000,0.584266,0.173614,13356903,3766660,1528011,544676,147550,18328,1727,113,334.762757,623733.428571,2.470887e+06,3.176884e+11,1.608139e+07,9.438033e+06,276.662666,1744.273829,316.476519,2050.000,117.508697,0.054214,2167.508697,363271,0.004511,0.995489,437.986058,496.426666,770.608582,0.589576,2167.689337,6608.142895,1.534916,35.065190,4139.98,4108.37,4184.69,3850.00,334.69,381.309763
2017-01-04,0.000000,0.584145,0.173561,13401104,3774377,1527463,544967,147581,18257,1735,115,339.910422,628880.857143,2.452330e+06,3.176884e+11,1.608330e+07,9.621572e+06,278.051907,1781.517871,318.274118,1912.500,193.337436,0.091810,2105.837436,394821,0.002730,0.997270,443.639716,520.359955,798.450417,0.600955,2178.315185,6619.800722,1.603303,56.936071,4086.29,4139.98,4211.08,4032.62,178.46,467.083022
2017-01-05,0.000000,0.582931,0.173584,13428384,3788135,1532571,546811,148115,18296,1720,116,342.573106,628369.428571,2.429770e+06,3.176884e+11,1.608505e+07,1.010663e+07,280.917262,1805.569077,316.979453,1750.000,114.991943,0.061658,1864.991943,352053,0.032765,0.967235,447.359942,531.094509,803.477250,0.631083,2200.523544,6629.752571,1.782596,218.342014,4016.00,4086.29,4119.62,3911.79,207.83,691.743060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-26,0.168700,0.657364,0.310983,54313850,12680959,4580740,1013552,152671,16050,2081,102,24225.667582,732684.571429,6.407517e+08,8.840440e+13,1.975937e+07,1.069032e+07,25223.088951,142622.066005,24908.803530,406.250,8.530836,0.020567,414.780836,311475,0.069042,0.930958,31884.744000,22249.997332,62532.881992,0.541627,160840.604782,402855.709966,0.613098,146.820920,65173.99,63152.01,65839.00,62670.00,3169.00,28373.305930
2024-09-27,0.168708,0.657467,0.311115,54345381,12682492,4580649,1013559,152640,16049,2081,102,24254.574787,723668.714286,6.313353e+08,8.840440e+13,1.975980e+07,1.075623e+07,25235.339742,142781.740702,24918.226512,428.125,11.071403,0.025208,439.196403,324316,0.060524,0.939476,31917.205174,23771.569581,62658.791082,0.544960,160915.238123,403193.477589,0.618093,87.539794,65769.95,65173.99,66498.00,64819.90,1678.10,22048.804870
2024-09-28,0.168819,0.657537,0.311122,54393608,12685210,4580945,1013696,152656,16047,2081,103,24276.368512,720212.285714,6.234776e+08,8.840440e+13,1.976020e+07,1.076734e+07,25240.291974,142875.916723,24